In [0]:
# Code to read csv file into colaboratory:
#STEP-1: Import Libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
#STEP-2: Autheticate E-Mail ID
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#STEP-3: Get File from Drive using file-ID
downloaded = drive.CreateFile({'id':'1RcJIJrL_OMdtnntYUxWXD0cIgshNX2Ct'}) # replace the id with id of file you want to access
downloaded.GetContentFile('data.csv') 

In [0]:
#STEP-4: Read File
#Load data and store it in the format n_samples*nfeatures(Required format for the kMeans algorithm)

from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import csv
k1= 8

with open('data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    #allows to skip the first row
    next(readCSV, None)
    input_list = []
    stock_names = []
    for row in readCSV:
        stock_names.append(row[0])
        row = row[1:len(row)]
        input_list.append(row)
input = np.asarray(input_list)

In [71]:
#Question 1:
#in this question we will initialize the parameters using the default values
kmeans = KMeans(init='random', n_clusters=k1, max_iter=1000, n_init=100).fit(input)
print("the inertia with %d centroids is %f :"% (k1,kmeans.inertia_))

the inertia with 8 centroids is 1590.136410 :


In [72]:
#Question 2 Here i'm going to modify the following parameters :INITIALIZATION OF CENTROIDS and the NUMBER OF ITERATIONS
  #2.a)INITIALIZATION OF CENTROIDS: The initialization of the centroids can directly affect the quality of the clutering and thefore
  #also the SSE. In the previous run, the centroids where initialized randomly without using any "smart" criteria;in the follwing run we will initialize
  #them using the kMeans approach which will try to intialize the centroids so that points assigned to the same cluster are similar (in some sense). As a consequence
  #to this change, the SSE should be in the worst case equal to the one obtained with a random initialization(this may happend only if the random centroids
  # coincide with the ones of the kMeans++ since all the other parameters are equal) or in genral better that the previous one.

kmeans = KMeans(init='k-means++', n_clusters=k1, max_iter=1000, n_init=100).fit(input)
print("the inertia with %d centroids when initalizing them with the kmeans++  algorithm and the same number of iterations as before is %f "% (k1,kmeans.inertia_))


the inertia with 8 centroids when initalizing them with the kmeans++  algorithm and the same number of iterations as before is 1533.989491 


In [73]:
#2.b) NUMBER OF ITERATIONS : We have a lemma that says "the SSE always decreases", so increasing the number of iterations will for sure decrease
  #the value of the SSE and therefore producing a better clustering. Beyond initializing the centroids with K-Means++,we will also increase the number
  #of iterations from 100 to 500.

kmeans = KMeans(init='k-means++', n_clusters=k1, max_iter=10000, n_init=500).fit(input)
print("the inertia with %d centroids after increasing the number of iterations is %f :"% (k1,kmeans.inertia_))


the inertia with 8 centroids after increasing the number of iterations is 1519.403917 :


In [74]:
#Question 3: LABELING EACH LCUSETER OBTAINED

labels = kmeans.labels_
#label_elements = [object] * 8
label_elements = {}

#index represent the stock index
index =0

for label in labels:
    if label_elements.__contains__(label) :
        #get the current list
        elements = label_elements.get(label)
        elements.append(stock_names[index])
        label_elements.__setitem__(label, elements)
    else:
        elements = [stock_names[index]]
        label_elements.__setitem__(label, elements)
    index = index+1


for classe in range(k1):
    print("cluster number and items :", (classe, label_elements[classe]))

#cluster 0: Label = FOOD
  #becuase we have Procter & Gamble,Kraft,McDonalds that deal with food while the other ones are independent; we may also include Coca cola
#cluster 1: Label = TRANSPORT
 # because we AMERICAN EXPRESS,BOING, while the other ones are independent.
#cluster 2: Label = ?
  # difficult to classify because each stock deals with a different sector
#cluster 3: Label = TECHNOLOGY 
  #it's basically an epmty clisyer and the singlo elent deals with technology
#cluster 4: Label = ?
 #difficult to classify because each stock deals with a different sector
#cluster 5: Label = TECHNOLOGY
  #it's basically an empty cluster and the only element insdie deals with technologies
#cluster 6: Label = BANK
  #because bank of america is the only element inside
#cluster 7: Label = TECHNOLOGY
  #becaseu intel which is the ony element inside deals with TECHNOLOGY

cluster number and items : (0, ['Kraft', 'Verizon', 'IBM', 'The Home Depot', 'Procter & Gamble', 'Wal-Mart', 'AT&T', 'Merck', 'McDonalds', 'Coca-Cola'])
cluster number and items : (1, ['American Express', 'Boeing', 'Microsoft', 'Walt Disney', 'General Electric', 'United Technologies', 'Travelers', 'JPMorgan Chase', '3M', 'Johnson & Johnson'])
cluster number and items : (2, ['Chevron', 'Pfizer', 'ExxonMobil'])
cluster number and items : (3, ['Hewlett-Packard'])
cluster number and items : (4, ['DuPont', 'Caterpillar', 'Alcoa'])
cluster number and items : (5, ['Cisco Systems'])
cluster number and items : (6, ['Bank of America'])
cluster number and items : (7, ['Intel'])
